# 집 근처 주유소 10곳 조사

In [1]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### opinet에서 인근 구의 주유소 목록 불러오기

In [3]:
driver = webdriver.Chrome('G:\Workspace\Data-Analysis\Data-Analysis\chromedriver.exe')
driver.get('https://www.opinet.co.kr/user/main/mainView.do')
time.sleep(1)

driver.find_element_by_css_selector('.ic_m1').click()
gu_nearby = ['양천구', '강서구']

for gu in gu_nearby:
    region = driver.find_element_by_id("SIGUNGU_NM0")
    region.send_keys(gu)
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()
    time.sleep(2)

In [4]:
driver.quit()

### 주유소 리스트 정리

In [5]:
from glob import glob
station_files = glob('./station_data/지역*.xls')
tmp_raw = []

for file_name in station_files:
    tmp = pd.read_excel(file_name, header=2) #맨 처음 두 줄은 건너뜀
    tmp_raw.append(tmp)

station_raw = pd.concat(tmp_raw)

In [6]:
station_list = pd.DataFrame({
    '이름': station_raw['상호'],
    '주소': station_raw['주소'],
    '가격': station_raw['휘발유']
})
station_list['구'] = [addr.split()[1] for addr in station_list['주소']]
yangcheon_station = station_list.loc[station_list['구'] == '양천구']
gangseo_station = station_list.loc[station_list['구'] == '강서구']
nearby_station = yangcheon_station.append(gangseo_station).reset_index()
del nearby_station['index']

### 카카오맵 api에서 위도, 경도 불러오기

In [7]:
key_fd = open('../06. RestAPI/kakaokey.txt', mode='r')
kakao_key = key_fd.read(100)
key_fd.close()

In [8]:
addr_list = nearby_station['주소']
lat_list = []; lng_list = []

In [9]:
for addr in addr_list:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(addr)
    result = requests.get(url, headers={"Authorization": 'KakaoAK '+kakao_key}).json()
    lng = float(result['documents'][0]['x'])
    lat = float(result['documents'][0]['y'])
    lat_list.append(lat)
    lng_list.append(lng)

In [10]:
stations = pd.DataFrame({
    '주유소': nearby_station['이름'],
    '주소': nearby_station['주소'],
    '가격': nearby_station['가격'],
    '위도': lat_list,
    '경도': lng_list
})

### 가격 순 정렬

In [11]:
stations = stations.sort_values(by='가격').head(10).reset_index()
del stations['index']

### 지도에 그리기

In [12]:
import folium

In [13]:
mapping = folium.Map(location=[stations['위도'].mean(), stations['경도'].mean()], zoom_start=14)
for i in stations.index:
    if i == 0:
        color = 'red'
    else:
        color = 'green'
    folium.Marker(
        location=[stations['위도'][i], stations['경도'][i]],
        popup=stations['주유소'][i],
        icon=folium.Icon(color=color),
        tooltip=f'{stations["주유소"][i]}, 가격: {stations["가격"][i]}'
    ).add_to(mapping)
mapping